In [32]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [95]:
import gc

In [3]:
# Create path
path =  r'C:\Users\Marker\Videos\Instakart Basket Analysis 01-25'

In [5]:
# Import ords_prods_merge dataframe 
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [7]:
df_ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,7.0,10,0,both
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,both
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,2,0,both
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.0,1,0,both
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.0,1,0,both
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.0,2,1,both


In [9]:
df_ords_prods_merge.shape

(32404859, 14)

In [13]:
# Define a function for "price label"
def price_label(row): 
    if row['prices'] <= 5: return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15): return 'Mid-range product'
    elif row['prices'] > 15:  return 'High range product'
    else: return 'Not enough data'

In [15]:
df_ords_prods_merge.apply(price_label, axis=1)

0           Mid-range product
1           Mid-range product
2           Mid-range product
3           Mid-range product
4           Mid-range product
                  ...        
32404854    Mid-range product
32404855    Mid-range product
32404856    Mid-range product
32404857    Mid-range product
32404858    Mid-range product
Length: 32404859, dtype: object

In [34]:
# Create a subset of first million rows
df = df_ords_prods_merge[:1000000]

In [36]:
df.shape

(1000000, 14)

In [44]:
# Use loc() method on df subset to apply price range function
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [40]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [42]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
NaN                   652638
Low-range product     338018
High-range product      9344
Name: count, dtype: int64

In [46]:
# Use loc() method on entire ords_prods_merge dataframe to apply price range function
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [47]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [48]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [49]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   product_id              1000000 non-null  int64   
 1   product_name            1000000 non-null  object  
 2   aisle_id                1000000 non-null  int64   
 3   department_id           1000000 non-null  int64   
 4   prices                  1000000 non-null  float64 
 5   order_id                1000000 non-null  int64   
 6   user_id                 1000000 non-null  int64   
 7   order_number            1000000 non-null  int64   
 8   order_dow               1000000 non-null  int64   
 9   order_hour_of_day       1000000 non-null  int64   
 10  days_since_prior_order  1000000 non-null  float64 
 11  add_to_cart_order       1000000 non-null  int64   
 12  reordered               1000000 non-null  int64   
 13  _merge                  1000000 non-null  c

In [54]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

In [56]:
# Create "busiest_day" column
# Find frequency of orders_day_of_the_week
df_ords_prods_merge['order_dow'].value_counts(dropna = False)

order_dow
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [59]:
# Use for-loop to determine whether orders are on "busiest day" (0 = Saturday), "least busy" (4 = Wednesday), or "regularly busy" (other days of the week)
result = []
for value in df_ords_prods_merge["order_dow"]:
  if value == 0: result.append("Busiest day")
  elif value == 4: result.append("Least busy")
  else: result.append("Regularly busy")

In [60]:
result

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Bus

In [61]:
# Create new column in ords_prods_merge to view results in context
df_ords_prods_merge['Busiest_day'] = result

In [65]:
df_ords_prods_merge['Busiest_day'].value_counts(dropna = False)

Busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

In [67]:
#Step 2
# Create column “busiest_days" from "order_dow" to identify two busiest days (0, 1), slowest days (4, 3), or regularly busy (all other days).

result_2 = []
for value in df_ords_prods_merge["order_dow"]:
  if value == 0 or value == 1:  result_2.append("Busiest days")
  elif value == 4 or value == 3: result_2.append("Slowest days")
  else: result_2.append("Regularly busy")

In [68]:
result_2

['Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',

In [71]:
#Step 3 Check values of new "busiest days" column for accuracy with observations in markdown format
# Create new 'busiest_days' column in ords_prods_merge to view results in context.
df_ords_prods_merge['Busiest_days'] = result_2

In [73]:
df_ords_prods_merge['Busiest_days'].value_counts(dropna = False)

Busiest_days
Regularly busy    12916111
Busiest days      11864412
Slowest days       7624336
Name: count, dtype: int64

In [75]:
df_ords_prods_merge.head ()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,Busiest_day,Busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,7.0,10,0,both,Mid-range product,Regularly busy,Slowest days
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Slowest days


The total value counts for the ords_prods_merge dataframe is equal to the total sum of all groupings (i.e. "Regularly busy" + "Busiest days" + "Slowest days"). In addition, the listed sums for each label match the sum totals for the qualifying days of the week (e.g. Total "Busiest days" = 11864412, which matches orders_day_of_week 0 + 1.

In [79]:
#Step 4 Create new column "busiest_period_of_day" to identify time periods “Most orders” “Average orders” and “Fewest orders”
#Check value counts in "order_hour_of_day" column
df_ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

The value counts listed above will be split into equal thirds for the following labels: "Most orders" = 10, 11, 14, 15, 13, 12, 16, and 9; "Fewest orders" = 23, 6, 0, 1, 5, 2, 4, and 3; "Average orders" = 17,8,18,19,20,7,21 and 22.

In [81]:
# Create for-loop if statement labeling periods of time as “Most orders,” “Average orders,” and “Fewest orders.”
result_3 = []
for value in df_ords_prods_merge["order_hour_of_day"]:
  if value in [10, 11, 14, 15, 13, 12, 16, 9]: result_3.append("Most orders")
  elif value in [23, 6, 0, 1, 5, 2, 4, 3]: result_3.append("Fewest orders")
  else: result_3.append("Average orders")

In [82]:
result_3


['Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most ord

In [83]:
# Create new column "busiest_period_of_day" in ords_prods_merge.
df_ords_prods_merge['busiest_period_of_day'] = result_3

In [89]:
# Step 5 Print frequency of "busiest_period_of_day" column
# Print value counts in "busiest_period_of_day" column.
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: count, dtype: int64

In [91]:
df_ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 18 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   order_dow               int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  price_range_loc         object  
 15  Busiest_day             object  
 16  Busiest_days            object  
 17  busiest_period_of_day   object  
dtypes: category(1), float64(2), int64(10), object(5)
memory usage: 4.1+ GB


In [93]:
#Step 7 Export dataframe as a pickle file to “Prepared Data” folder
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_refined.pkl'))